Installing necessary prereqs: [NetworkX](https://networkx.org/documentation/stable/reference/index.html) for graphing, [PuLP](https://coin-or.github.io/pulp/main/index.html) for linear programming.

In [1]:
%pip install networkx pulp

Note: you may need to restart the kernel to use updated packages.


In [8]:

import json
import networkx as nx

file = open("map_ct_adjusted.json")
data = json.load(file)

adjacencies = data["adj"]
# TODO: Shits the bed if there's anything other than letters and _
countyNames = data["GEOID"]
pop = data["pop"]


# Number of districts
NUM_DISTRICTS = 5

# Population Tolerance
ALPHA = .05

In [9]:
from random import Random

stateGraph = nx.Graph()

r =  Random(51535162)
totalPop = 0
# Add the nodes as necessary

for i in range(len(adjacencies)):
    # Adds node for each set of adjacencies
    totalPop += pop[i]
    # Adds necessary edges; double-adding doesn't matter
    try:
        for j in adjacencies[i]:
            # Add edge with uniformly random weight 0, 10
            stateGraph.add_edge(countyNames[i], countyNames[j], weight=r.uniform(0, 10))
    except:
            stateGraph.add_edge(countyNames[i], countyNames[adjacencies[i]], weight=r.uniform(0, 10))


# nx.draw(stateGraph)

In [10]:
# Population per district
targetPop = totalPop / NUM_DISTRICTS

# Binary indicators, defined as 1 if in node in same partitition as 
# source S or 0 in same as sink O. Initializes to 0.
V = [0 for i in range(stateGraph.order())]

# 
Y = [[0 for i in range(stateGraph.order())] for j in range(stateGraph.order())]


In [11]:
import pulp 

def CutPartition(remainingGraph):
    prog = pulp.LpProblem("CutPartition", pulp.LpMinimize)

    # Defining Partition Binary
    xVars = []
    xVarMap = {}
    for n in remainingGraph.nodes:
        var = pulp.LpVariable("x" + str(n), cat=pulp.LpBinary)
        xVarMap[n] = var
        xVars.append(var)
        prog += var
    # Defining crossing binary
    edgeVars = []
    weights = []
    for n, m in remainingGraph.edges:
        edgeVal = pulp.LpAffineExpression([(xVarMap[n], 1), (xVarMap[m], -1)])
        absEdgeVal = pulp.LpVariable('y' + str(n) + str(m), cat=pulp.LpBinary)

        # Tries to fix absolute value prob
        prog += absEdgeVal >= edgeVal
        prog += absEdgeVal >= -edgeVal

        edgeVars.append(absEdgeVal)
        weights.append(remainingGraph[n][m]['weight'])
    # Subset sum
    popVariables = []
    for i in range(len(xVars)):
        popVariables.append(pulp.LpAffineExpression(xVars[i] * pop[i]))
    # Max Pop
    prog += pulp.lpSum(popVariables) <= targetPop * (1 + ALPHA)
    # Min Pop
    prog += pulp.lpSum(popVariables) >= targetPop * (1 - ALPHA)

    # Define Minimization Target
    # Edgeweights
    edgeWeightVars = []
    for i in range(len(edgeVars)):
        edgeWeightVars.append(pulp.LpAffineExpression(edgeVars[i] * weights[i]))
    prog += pulp.LpAffineExpression(pulp.lpSum(edgeWeightVars), name="Z")

    prog.solve()

    toReturn = {}

    # districtPop = 0
    for i in range(len(xVars)):
        if(xVars[i].value() == 1):
            # districtPop += pop[i]
            toReturn[xVars[i].name] = xVars[i].value()
            # print(xVars[i].name, xVars[i].value())
    # print(districtPop)
    return toReturn


remainingGraph = stateGraph.copy()
districts = []
while len(districts) < NUM_DISTRICTS - 1:
    cutResult = CutPartition(remainingGraph)
    newDistrict = []
    for d in cutResult.keys():
        # Gets rid of the x
        # print(d)
        newDistrict.append(d[1:])
    # Appends the new district to our list of districts
    districts.append(newDistrict)
    oldSize = remainingGraph.order()
    # Removes those nodes from the district
    remainingGraph.remove_nodes_from(newDistrict)
    newSize = remainingGraph.order()
    # Verify that all of the nodes were removed
    if(newSize != oldSize - len(newDistrict)):
        print(remainingGraph.nodes)
        print(newDistrict)
        raise(ValueError("Districts were not removed appropriately"))
    # print(newDistrict)
newDistrict = []
for d in remainingGraph.nodes:
    # Gets rid of the x
    newDistrict.append(d)
districts.append(newDistrict)

#Confirm total size is appropriate
totalCountiesinDistricts = 0
for d in districts:
    totalCountiesinDistricts += len(d)
if(totalCountiesinDistricts != len(countyNames)):
    raise(ValueError("Number of districts in plan does not match number of districts."))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z3/3qb7cv41693fmz3b6hbntjkr0000gn/T/da2352ad001248d49b5c5e72559594c7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/z3/3qb7cv41693fmz3b6hbntjkr0000gn/T/da2352ad001248d49b5c5e72559594c7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4281 COLUMNS
At line 26575 RHS
At line 30852 BOUNDS
At line 33756 ENDATA
Problem MODEL has 4276 rows, 2903 columns and 14350 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.05 seconds
Cgl0004I processed model has 4275 rows, 2903 columns (2903 integer (2903 of which binary)) and 13586 elements
Cbc0038I Initial state - 766 integers unsatisfied sum - 145.54
Cbc0038I Pass   1: (0.22 seconds) suminf.  119.978

In [12]:
import csv

file = open('ct_districts.csv', 'w')
w = csv.writer(file)
w.writerow(["district", "precinct"])
for i in range(len(districts)):
    for j in range(len(districts[i])):
        w.writerow([i + 1, districts[i][j]])
file.close()